In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

In [4]:
pd.set_option('display.max_columns', 500)
raw = pd.read_csv('../data/MULTI_trust_politicians_eu_9_raw.csv')

In [5]:
raw.head()

,essround,idno,cntry,trstprl,trstplt,trstep,euftf,Unnamed: 7
0,9,17,AT,5,5,5,10,NaN
1,9,31,AT,5,3,2,2,NaN
2,9,58,AT,5,5,5,5,NaN
3,9,69,AT,2,2,2,2,NaN
4,9,98,AT,7,3,5,5,NaN


In [3]:
countries = ['DE','FR','IT','PL','HU']

In [6]:
raw = raw[raw['cntry'].isin(countries)].drop(columns='Unnamed: 7').reset_index(drop=True)

In [7]:
raw.head()

,essround,idno,cntry,trstprl,trstplt,trstep,euftf
0,9,1,DE,4,5,7,9
1,9,63,DE,6,6,7,6
2,9,108,DE,88,88,88,88
3,9,117,DE,6,5,6,7
4,9,134,DE,8,5,5,10


In [8]:
cols = {'essround':'round', 
        'cntry':'country', 
        'trstprl':'t_parlament',
        'trstplt':'t_politicians',
        'trstep':'t_eu_par', 
        'euftf':'eu_further'}

In [9]:
tr_9 = raw.rename(columns=cols)

In [10]:
tr_9.head()

,round,idno,country,t_parlament,t_politicians,t_eu_par,eu_further
0,9,1,DE,4,5,7,9
1,9,63,DE,6,6,7,6
2,9,108,DE,88,88,88,88
3,9,117,DE,6,5,6,7
4,9,134,DE,8,5,5,10


#### t_parlament: Trust in country´s parliament
##### 0 (not at all) - 10 (complete trust), 77(refusal), 88(don´t know), 99(no answer)
#### t_politicians: Trust in politicians
##### 0 (not at all) - 10 (complete trust), 77, 88, 99
#### t_eu_par: Trust in the European Parliament
##### 0 (not at all) - 10 (complete trust), 77, 88, 99
#### eu_further: European unification go further or gone too far
##### 0 (already gone too far) - 10 (unification go further), 77, 88, 99

In [11]:
tr_9.shape

(10311, 7)

In [12]:
tr_9 = tr_9[(~tr_9['t_parlament'].isin([77,88,99]))
      &(~tr_9['t_politicians'].isin([77,88,99]))
      &(~tr_9['t_eu_par'].isin([77,88,99]))
      &(~tr_9['eu_further'].isin([77,88,99]))]

In [13]:
tr_9.shape

(9206, 7)

In [61]:
tr_9 = tr_9[['country','round','idno','t_parlament','t_politicians','t_eu_par','eu_further']]

In [66]:
tr_9[['eu_further']] = tr_9[['eu_further']].astype(float)

In [67]:
tr_9.dtypes

country           object
round              int64
idno               int64
t_parlament        int64
t_politicians      int64
t_eu_par           int64
eu_further       float64
dtype: object

In [15]:
# Now we do the same with our df with historical data

In [16]:
raw_h = pd.read_csv('../data/MULTI_trust_politicians_eu_historic_raw.csv')

In [17]:
raw_h.columns

Index(['cntry', 'cname', 'cedition', 'cproddat', 'cseqno', 'name', 'essround',
       'edition', 'idno', 'dweight', 'pspwght', 'pweight', 'trstprl',
       'trstplt', 'trstep', 'euftf'],
      dtype='object')

In [18]:
raw_h = raw_h.drop(columns={'cname', 
                            'cedition', 
                            'cproddat', 
                            'cseqno', 
                            'name',
                            'edition',
                            'dweight', 
                            'pspwght', 
                            'pweight'})

In [19]:
raw_h.head()

,cntry,essround,idno,trstprl,trstplt,trstep,euftf
0,DE,1,101114,0,0,7,NaN
1,DE,1,101120,7,4,88,NaN
2,DE,1,101126,4,4,5,NaN
3,DE,1,101304,6,2,5,NaN
4,DE,1,101322,3,1,5,NaN


In [21]:
raw_h.isna().sum()

cntry           0
essround        0
idno            0
trstprl         0
trstplt         0
trstep          0
euftf       17495
dtype: int64

In [57]:
#NOTE: The euftf (european union: unification further?) variable wasn´t included in the 1st and 5th rounds
raw_h[raw_h['essround']==1].isna().sum()

cntry          0
essround       0
idno           0
trstprl        0
trstplt        0
trstep         0
euftf       9424
dtype: int64

In [59]:
tr_h = raw_h.rename(columns=cols)

In [60]:
tr_h.head()

,country,round,idno,t_parlament,t_politicians,t_eu_par,eu_further
0,DE,1,101114,0,0,7,NaN
1,DE,1,101120,7,4,88,NaN
2,DE,1,101126,4,4,5,NaN
3,DE,1,101304,6,2,5,NaN
4,DE,1,101322,3,1,5,NaN


In [63]:
tr_h.dtypes

country           object
round              int64
idno               int64
t_parlament        int64
t_politicians      int64
t_eu_par           int64
eu_further       float64
dtype: object

In [68]:
tr_h.shape

(70442, 7)

In [69]:
tr_h = tr_h[(~tr_h['t_parlament'].isin([77,88,99]))
      &(~tr_h['t_politicians'].isin([77,88,99]))
      &(~tr_h['t_eu_par'].isin([77,88,99]))
      &(~tr_h['eu_further'].isin([77,88,99]))]

In [70]:
tr_h.shape

(61403, 7)

### Now we concatenate the two dfs to have one single df with values for our variables over time (2002-2018)

In [71]:
tr_eu = pd.concat([tr_h, tr_9]).sort_values(['round','country']).reset_index(drop=True)

In [72]:
tr_eu.head()

,country,round,idno,t_parlament,t_politicians,t_eu_par,eu_further
0,DE,1,101114,0,0,7,NaN
1,DE,1,101126,4,4,5,NaN
2,DE,1,101304,6,2,5,NaN
3,DE,1,101322,3,1,5,NaN
4,DE,1,101405,3,4,7,NaN


In [73]:
tr_eu.shape

(70609, 7)

In [74]:
tr_eu.dtypes

country           object
round              int64
idno               int64
t_parlament        int64
t_politicians      int64
t_eu_par           int64
eu_further       float64
dtype: object

In [75]:
tr_eu.to_csv('../data/trust_politicians_eu_multi_2002_2018_DF.csv', index=False)